In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import gensim
from gensim.parsing.preprocessing import remove_stopwords

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

from scipy.stats.stats import pearsonr   
import numpy as np

import en_core_web_sm
import en_core_web_lg

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import time
import string
import random

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import spacy

from gensim.parsing.preprocessing import remove_stopwords

from keras.models import Sequential
from keras.models import Model
from keras.layers import (Dense, Embedding, Reshape, Activation, 
                          SimpleRNN, GRU, LSTM, Convolution1D, 
                          MaxPooling1D, concatenate, Dropout, 
                          Conv1D, Flatten, BatchNormalization,
                         Lambda, SpatialDropout1D)
from keras import regularizers
from keras.preprocessing.sequence import pad_sequences
from keras.regularizers import L1L2
from keras import Input
from keras import backend as K
from keras.layers.wrappers import Bidirectional
from keras import callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint

from scipy.sparse import hstack


Using TensorFlow backend.


In [3]:
all_qs = pd.read_csv('train.csv')

In [4]:
all_qs = all_qs.iloc[:100000]

In [5]:
#pd.set_option('max_colwidth', 100)
pd.set_option('display.max_colwidth', -1)

In [6]:
all_qs.head(100)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?,"I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone and video games?,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Motorolla DCX3400?,How do I hack Motorola DCX3400 for free internet?,0


In [7]:
lengths = all_qs['question1'].str.len()
print(lengths.max())

623


In [8]:
all_qs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
id              100000 non-null int64
qid1            100000 non-null int64
qid2            100000 non-null int64
question1       100000 non-null object
question2       100000 non-null object
is_duplicate    100000 non-null int64
dtypes: int64(4), object(2)
memory usage: 4.6+ MB


In [9]:
all_qs.describe()

,id,qid1,qid2,is_duplicate
count,100000.000000,100000.00000,100000.000000,100000.000000
mean,49999.500000,76207.29047,76204.708200,0.372540
std,28867.657797,48355.48207,48454.311214,0.483483
min,0.000000,1.00000,2.000000,0.000000
25%,24999.750000,33292.00000,32946.750000,0.000000
50%,49999.500000,72895.00000,72756.000000,0.000000
75%,74999.250000,117412.50000,117562.750000,1.000000
max,99999.000000,165930.00000,165931.000000,1.000000


In [10]:
X = all_qs[['question1','question2']]

In [11]:
y = all_qs[['is_duplicate']]

In [12]:
q1s = list(X['question1'])

In [13]:
q1s

['What is the step by step guide to invest in share market in india?',
 'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
 'How can I increase the speed of my internet connection while using a VPN?',
 'Why am I mentally very lonely? How can I solve it?',
 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?',
 'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
 'Should I buy tiago?',
 'How can I be a good geologist?',
 'When do you use シ instead of し?',
 'Motorola (company): Can I hack my Charter Motorolla DCX3400?',
 'Method to find separation of slits using fresnel biprism?',
 'How do I read and find my YouTube comments?',
 'What can make Physics easy to learn?',
 'What was your first sexual experience like?',
 'What are the laws to change your status from a student visa to a green card in the US, how do they compare to the immigration laws in Canada?',
 'What would a Trump presidency mean for current international 

In [14]:
q2s = list(X['question2'])

In [15]:
no_sw_q1s = [t.lower() for t in q1s ]

In [16]:
no_sw_q1s

['what is the step by step guide to invest in share market in india?',
 'what is the story of kohinoor (koh-i-noor) diamond?',
 'how can i increase the speed of my internet connection while using a vpn?',
 'why am i mentally very lonely? how can i solve it?',
 'which one dissolve in water quikly sugar, salt, methane and carbon di oxide?',
 'astrology: i am a capricorn sun cap moon and cap rising...what does that say about me?',
 'should i buy tiago?',
 'how can i be a good geologist?',
 'when do you use シ instead of し?',
 'motorola (company): can i hack my charter motorolla dcx3400?',
 'method to find separation of slits using fresnel biprism?',
 'how do i read and find my youtube comments?',
 'what can make physics easy to learn?',
 'what was your first sexual experience like?',
 'what are the laws to change your status from a student visa to a green card in the us, how do they compare to the immigration laws in canada?',
 'what would a trump presidency mean for current international 

In [17]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        

In [18]:
no_sw_q1s = list(sent_to_words(no_sw_q1s))

In [19]:
no_sw_q1s[0]

['what',
 'is',
 'the',
 'step',
 'by',
 'step',
 'guide',
 'to',
 'invest',
 'in',
 'share',
 'market',
 'in',
 'india']

In [20]:
no_sw_q1s = [' '.join(words) for words in no_sw_q1s]

In [21]:
no_sw_q1s[5]

'astrology am capricorn sun cap moon and cap rising what does that say about me'

In [22]:
nlp2 = en_core_web_lg.load()

In [23]:
%%time
q1_subj = []
q1_root = []
what_list1 = []
for index, q in enumerate(q1s):
    
    doc2 = nlp2(q)         
    
    subj = ''
    root = ''
    for word in doc2:
        if word.dep_ == 'nsubj':
            subj = word
            
        elif word.dep_ == 'ROOT':
            root = word
            
    q1_subj.append(subj)
    q1_root.append(root)
    
    ne_list = [token.text.lower() for token in doc2.ents]

    no_sw_q1s[index] = no_sw_q1s[index] + ' ' + ' '.join(ne_list) + ' ' + ' '.join(ne_list)
        
    what = ''
    for token in doc2:
        if token.tag_ == 'WP' or token.tag_ == 'WRB':
            what = token
            break
    what_list1.append(what)     
  

CPU times: user 25min 27s, sys: 1min 41s, total: 27min 8s
Wall time: 14min 22s


In [24]:
spacy.explain('WP')

'wh-pronoun, personal'

In [25]:
%%time
similarity = []
for index, q in enumerate(q1s):
    
    doc1 = nlp2(q)  
    doc2 = nlp2(q2s[index])
    similarity.append(doc1.similarity(doc2))


CPU times: user 50min 1s, sys: 3min 13s, total: 53min 15s
Wall time: 27min 49s


In [26]:
similarity

[0.9866842502321557,
 0.9249995494188401,
 0.9290680453645342,
 0.7307424309555903,
 0.8466158927439796,
 0.9499860921536979,
 0.6403895318555238,
 0.9807831604694341,
 0.8788559191910189,
 0.8559528995742688,
 0.7386789280319982,
 0.9804494257604397,
 0.9958758427217597,
 0.9931289461125838,
 0.996409017232944,
 0.910512380492492,
 0.9778671984309173,
 0.9466555486420116,
 0.9485820604053475,
 0.9697624276421036,
 0.8922699307395492,
 0.9564411541219144,
 0.9492066336196522,
 0.8359927119156265,
 0.9290005400542861,
 0.9891881203335958,
 0.9620694013606318,
 0.931147694315111,
 0.9886185096324566,
 0.9466545544600022,
 0.9775956278904575,
 0.9215169769827625,
 0.9835013011468509,
 0.7423219396276748,
 0.9687655929454554,
 0.9197562450135285,
 0.951303447083793,
 0.9588447378810063,
 0.8889260193092519,
 0.8670681676338755,
 0.7018312520048097,
 0.9890364932687851,
 0.9982024085435584,
 0.9375424279250472,
 0.9920916092538288,
 0.9280265374695484,
 0.48668552405881765,
 0.9313827239145

In [27]:
X['spacy_sim'] = np.asarray(similarity)

/anaconda3/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
no_sw_q1s

['what is the step by step guide to invest in share market in india india india',
 'what is the story of kohinoor koh noor diamond kohinoor koh-i-noor diamond kohinoor koh-i-noor diamond',
 'how can increase the speed of my internet connection while using vpn vpn vpn',
 'why am mentally very lonely how can solve it  ',
 'which one dissolve in water quikly sugar salt methane and carbon di oxide one one',
 'astrology am capricorn sun cap moon and cap rising what does that say about me  ',
 'should buy tiago  ',
 'how can be good geologist  ',
 'when do you use instead of  ',
 'motorola company can hack my charter motorolla dcx motorola charter motorolla dcx3400 motorola charter motorolla dcx3400',
 'method to find separation of slits using fresnel biprism  ',
 'how do read and find my youtube comments youtube youtube',
 'what can make physics easy to learn  ',
 'what was your first sexual experience like first first',
 'what are the laws to change your status from student visa to green c

In [29]:
#no_sw_q2s = [remove_stopwords(str(t).lower()) for t in q2s ]
no_sw_q2s = [t.lower() for t in q2s ]

In [30]:
no_sw_q2s = list(sent_to_words(no_sw_q2s))
no_sw_q2s = [' '.join(words) for words in no_sw_q2s]

In [31]:
%%time
q2_subj = []
q2_root = []
what_list2 = []
for index, q in enumerate(q2s):
    
    doc2 = nlp2(q)      
    
    subj = ''
    root = ''
    for word in doc2:
        if word.dep_ == 'nsubj':
            subj = word
#            subj = str(word)
        elif word.dep_ == 'ROOT':
            root = word
    q2_subj.append(subj)
    q2_root.append(root)
    
    ne_list = [token.text.lower() for token in doc2.ents]

    no_sw_q2s[index] = no_sw_q2s[index] + ' ' + ' '.join(ne_list) + ' ' + ' '.join(ne_list)
    
    what = ''
    for token in doc2:
        if token.tag_ == 'WP' or token.tag_ == 'WRB':
            what = token
            break
    what_list2.append(what)        
        

CPU times: user 24min 55s, sys: 1min 43s, total: 26min 39s
Wall time: 13min 50s


In [32]:
type(q1_subj[0])

spacy.tokens.token.Token

In [33]:
%%time

subj = []
for index, word in enumerate(q1_subj):
    if word and q2_subj[index]:
        subj.append(word.similarity(q2_subj[index]))
    else:
        subj.append(0)

        
subj_series = pd.Series(data=subj)
X['equal_subj'] = subj_series
    
root = []
for index, word in enumerate(q1_root):
    
    if word and q2_root[index]:
        root.append(word.similarity(q2_root[index]))
    else:
        root.append(0)
    
root_series = pd.Series(data=root)
X['equal_root'] = root_series
    
whats = []
for index, word in enumerate(what_list1):
    if word and what_list2[index]:
        whats.append(word.similarity(what_list2[index]))
    else:
        whats.append(0)
    
what_series = pd.Series(data=whats)
X['sim_what'] = root_series    
    

/anaconda3/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CPU times: user 10.9 s, sys: 14.3 s, total: 25.2 s
Wall time: 47.9 s


In [34]:
X.head(20)

,question1,question2,spacy_sim,equal_subj,equal_root,sim_what
0,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0.986684,1.000000,1.000000,1.000000
1,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0.925000,0.257531,0.445086,0.445086
2,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0.929068,0.000000,0.889018,0.889018
3,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0.730742,0.000000,0.468864,0.468864
4,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0.846616,0.331026,0.279079,0.279079
5,Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?,"I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?",0.949986,0.672172,1.000000,1.000000
6,Should I buy tiago?,What keeps childern active and far from phone and video games?,0.640390,0.549420,0.172112,0.172112
7,How can I be a good geologist?,What should I do to be a great geologist?,0.980783,1.000000,0.573945,0.573945
8,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0.878856,1.000000,1.000000,1.000000
9,Motorola (company): Can I hack my Charter Motorolla DCX3400?,How do I hack Motorola DCX3400 for free internet?,0.855953,1.000000,1.000000,1.000000


In [35]:
no_sw_q2s

['what is the step by step guide to invest in share market  ',
 'what would happen if the indian government stole the kohinoor koh noor diamond back indian kohinoor koh-i-noor indian kohinoor koh-i-noor',
 'how can internet speed be increased by hacking through dns dns dns',
 'find the remainder when math math is divided by 24,23 24,23',
 'which fish would survive in salt water  ',
 'triple capricorn sun moon and ascendant in capricorn what does this say about me capricorn sun moon capricorn capricorn sun moon capricorn',
 'what keeps childern active and far from phone and video games  ',
 'what should do to be great geologist  ',
 'when do you use instead of and  ',
 'how do hack motorola dcx for free internet motorola dcx3400 motorola dcx3400',
 'what are some of the things technicians can tell about the durability and reliability of laptops and its components  ',
 'how can see all my youtube comments youtube youtube',
 'how can you make physics easy to learn  ',
 'what was your firs

In [36]:
print(type(no_sw_q1s))

<class 'list'>


In [37]:
no_sw_q1s[0:10]

['what is the step by step guide to invest in share market in india india india',
 'what is the story of kohinoor koh noor diamond kohinoor koh-i-noor diamond kohinoor koh-i-noor diamond',
 'how can increase the speed of my internet connection while using vpn vpn vpn',
 'why am mentally very lonely how can solve it  ',
 'which one dissolve in water quikly sugar salt methane and carbon di oxide one one',
 'astrology am capricorn sun cap moon and cap rising what does that say about me  ',
 'should buy tiago  ',
 'how can be good geologist  ',
 'when do you use instead of  ',
 'motorola company can hack my charter motorolla dcx motorola charter motorolla dcx3400 motorola charter motorolla dcx3400']

In [38]:
no_sw_q2s[0:10]

['what is the step by step guide to invest in share market  ',
 'what would happen if the indian government stole the kohinoor koh noor diamond back indian kohinoor koh-i-noor indian kohinoor koh-i-noor',
 'how can internet speed be increased by hacking through dns dns dns',
 'find the remainder when math math is divided by 24,23 24,23',
 'which fish would survive in salt water  ',
 'triple capricorn sun moon and ascendant in capricorn what does this say about me capricorn sun moon capricorn capricorn sun moon capricorn',
 'what keeps childern active and far from phone and video games  ',
 'what should do to be great geologist  ',
 'when do you use instead of and  ',
 'how do hack motorola dcx for free internet motorola dcx3400 motorola dcx3400']

In [39]:
combined_qs = no_sw_q1s + no_sw_q2s

In [40]:
tf = TfidfVectorizer()

In [41]:
tf.fit(combined_qs)
#cv.fit(combined_qs)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [42]:
tf_q1s = tf.transform(no_sw_q1s)
#tf_q1s = cv.transform(no_sw_q1s)

In [43]:
tf_q2s = tf.transform(no_sw_q2s)
#tf_q2s = cv.transform(no_sw_q2s)

In [44]:
tf_q1s

<100000x45161 sparse matrix of type '<class 'numpy.float64'>'
	with 994186 stored elements in Compressed Sparse Row format>

In [45]:
tf_q2s

<100000x45161 sparse matrix of type '<class 'numpy.float64'>'
	with 1008598 stored elements in Compressed Sparse Row format>

In [46]:
indices1 = tf_q1s.indices
data1 = tf_q1s.data
indptr1 = tf_q1s.indptr

In [47]:
len(data1)

994186

In [48]:
indices2 = tf_q2s.indices
data2 = tf_q2s.data
indptr2 = tf_q2s.indptr

In [49]:
#figure out how to get dot products between rows of 2 sparse matrices efficiently
    
indices1 = tf_q1s.indices
data1 = tf_q1s.data
indptr1 = tf_q1s.indptr

indices2 = tf_q2s.indices
data2 = tf_q2s.data
indptr2 = tf_q2s.indptr

mat_cos = []

for index in range(0, len(indptr1) - 1):
    row1data_index = indptr1[index]
    row2data_index = indptr2[index]
    sum = 0
    while row1data_index < indptr1[index + 1] and row2data_index < indptr2[index + 1]:
        
        row1col_index = indices1[row1data_index]
        row2col_index = indices2[row2data_index]
        
        if row1col_index == row2col_index:
            sum += data1[row1data_index] * data2[row2data_index]
            row1data_index += 1
            row2data_index += 1
#note order of increment ... within a row, columns are in reverse order ... !            
        elif row1col_index < row2col_index:
            row2data_index += 1
        elif row2col_index < row1col_index:
            row1data_index += 1
            
    mat_cos.append(sum)


In [50]:
len(mat_cos)

100000

In [51]:
mat_cos = np.asarray(mat_cos)

In [52]:
type(mat_cos)

numpy.ndarray

In [53]:
X['Dot'] = mat_cos

In [54]:
#analyzer = SentimentIntensityAnalyzer()

In [55]:
X_train,X_holdout,y_train,y_holdout = train_test_split(X,y,test_size=0.2)

In [56]:
X.head()

,question1,question2,spacy_sim,equal_subj,equal_root,sim_what,Dot
0,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0.986684,1.000000,1.000000,1.000000,0.874706
1,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0.925000,0.257531,0.445086,0.445086,0.887973
2,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0.929068,0.000000,0.889018,0.889018,0.086794
3,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0.730742,0.000000,0.468864,0.468864,0.000000
4,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0.846616,0.331026,0.279079,0.279079,0.240032


In [57]:
type(X_train)

pandas.core.frame.DataFrame

In [58]:
roc_auc_score(y, mat_cos)

0.7421173829891978

In [59]:
clf = DecisionTreeClassifier(random_state=0)

In [60]:
X_train.head()

,question1,question2,spacy_sim,equal_subj,equal_root,sim_what,Dot
60007,Do you like anal sex?,Do you love to practice anal sex?,0.961813,1.000000,0.657904,0.657904,0.696699
52748,"If you have never had long lasting friendships, how do you not let that affect your ability to make new friends?",How do you go about telling your boyfriend that you think his new friend has intentions beyond friendship?,0.948308,0.438365,0.755850,0.755850,0.154673
70916,What types of meat would a panda eat? Why?,What may happen to a panda if they eat meat?,0.961485,0.125718,0.710539,0.710539,0.655946
83092,What would be an apt song (female version Hindi melodious low scale) to sing in a stage show?,What are some good Hindi songs for a (female) singing audition?,0.927739,0.315436,0.746417,0.746417,0.522154
11353,Who are the best pest control operators in Bangalore?,Which are the best pest control services in Bangalore?,0.966425,0.512564,1.000000,1.000000,0.868493


In [61]:
train_predictors = X_train.drop(['question1', 'question2'], axis=1)

In [62]:
cross_val_score(clf, train_predictors, y_train, cv=10)

array([0.64704412, 0.64654418, 0.64554431, 0.636375  , 0.642875  ,
       0.6425    , 0.63825   , 0.63295412, 0.64245531, 0.64045506])

with no equal_subj
array([0.59281437, 0.636     , 0.598     , 0.616     , 0.624     ,
       0.608     , 0.604     , 0.622     , 0.626     , 0.59118236])

In [63]:
train_predictors.head()

,spacy_sim,equal_subj,equal_root,sim_what,Dot
60007,0.961813,1.000000,0.657904,0.657904,0.696699
52748,0.948308,0.438365,0.755850,0.755850,0.154673
70916,0.961485,0.125718,0.710539,0.710539,0.655946
83092,0.927739,0.315436,0.746417,0.746417,0.522154
11353,0.966425,0.512564,1.000000,1.000000,0.868493


In [64]:
holdout_predictors = X_holdout.drop(['question1', 'question2'], axis=1)

In [65]:
ab = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1,
                         random_state=0)

In [66]:
cross_val_score(ab, train_predictors, np.ravel(y_train), cv=10)

array([0.71416073, 0.7000375 , 0.70503687, 0.7       , 0.696875  ,
       0.697375  , 0.70525   , 0.70208776, 0.70296287, 0.69396175])

In [67]:
ab.fit(train_predictors, np.ravel(y_train))

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
          n_estimators=50, random_state=0)

In [68]:
y_holdout_pred = ab.predict(holdout_predictors)

In [69]:
print(confusion_matrix(y_holdout, y_holdout_pred))
print(f'accuracy: {accuracy_score(y_holdout, y_holdout_pred)}')
print(f'f1: {f1_score(y_holdout, y_holdout_pred)}')

[[9383 3140]
 [2856 4621]]
accuracy: 0.7002
f1: 0.6065100406877543


In [70]:
xgb = XGBClassifier()

In [71]:
cross_val_score(xgb, train_predictors, np.ravel(y_train), cv=10)

array([0.72115986, 0.71016123, 0.71028621, 0.7105    , 0.7015    ,
       0.70775   , 0.71375   , 0.7095887 , 0.70683835, 0.70208776])

In [72]:
xgb.fit(train_predictors, np.ravel(y_train))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [73]:
y_holdout_pred = xgb.predict(holdout_predictors)

In [74]:
print(confusion_matrix(y_holdout, y_holdout_pred))
print(f'accuracy: {accuracy_score(y_holdout, y_holdout_pred)}')
print(f'f1: {f1_score(y_holdout, y_holdout_pred)}')

[[9618 2905]
 [2929 4548]]
accuracy: 0.7083
f1: 0.6092431346282653


In [75]:
y_holdout_xgb_prob = xgb.predict_proba(holdout_predictors)

In [76]:
X_holdout_dup = X_holdout.copy()

X_holdout_dup['isDupe'] = y_holdout['is_duplicate']

X_holdout_dup['calledDupe'] = y_holdout_pred

X_holdout_dup.tail(50)

,question1,question2,spacy_sim,equal_subj,equal_root,sim_what,Dot,isDupe,calledDupe
309,Why and how did you break up with her/him?,How and why did you break up with your ex?,0.968921,1.000000,1.000000,1.000000,0.679543,1,1
33311,"Schools in Maine however, have made great efforts to improve the nutrition and palatability of the daily offering?","Imagine your friend has been offered $100,000. However, your friend has to offer you a portion of the $100K and you have to accept the amount in order for him/her to receive the rest. If he/she offers you $1, would you accept?",0.856568,0.218720,0.389998,0.389998,0.067983,0,0
52817,"Should I fully charge my newly purchased MacBook Air before the first use? If so, for how long?",How long time charge a new mobile before first use?,0.937098,0.444685,0.388855,0.388855,0.446666,0,0
75859,What are the best replies to the interview question 'where do you see yourself in the next 5 years'?,"How can you answer the ""where do you see yourself in 5 years"" question in a job interview when you KNOW you are going to start a business in 1 year?",0.972091,1.000000,0.372070,0.372070,0.508159,0,1
15517,What are the best selfies ever taken?,Which is your best selfie ever and why?,0.921878,0.000000,0.432354,0.432354,0.186934,0,0
99018,Can I downgrade from Windows 10 64bit to 32bit?,How can I downgrade from Windows 10 to Windows 7?,0.914400,1.000000,1.000000,1.000000,0.637678,0,0
16325,How France is seen in Vietnam and in Algeria ?,Why did France go to war against Vietnam?,0.872658,0.000000,0.476364,0.476364,0.687264,0,0
95494,"What are the diameters of Domino's large, medium and regular-sized pizzas?",How many calories does a Dominos pizza have?,0.797729,0.052303,0.734681,0.734681,0.000000,0,0
53550,"I'm repeatedly plagued by ankle injuries, while playing football, what would be the most effective rehab method?",Is playing American football worth the risk of injury?,0.879538,0.000000,0.352183,0.352183,0.147166,0,0
8862,I am a 2013 passout. My interest is in a software job. Which technology is good to get in a software company in Bangalore?,I am 31 years old high school passout. I dont have bachelor degree. If I learn computer programming will I get a job in software development company?,0.928338,0.129314,0.443714,0.443714,0.213504,0,0


In [77]:
X_holdout_both = X_holdout_dup[X_holdout_dup['isDupe'] ==  1]
X_holdout_both.tail(50)

,question1,question2,spacy_sim,equal_subj,equal_root,sim_what,Dot,isDupe,calledDupe
23952,What consequences would india go through if Donald trump wins?,What it means for India if Donald Trump win?,0.958681,1.000000,0.463096,0.463096,0.707093,1,1
38898,What is Tianjin famous for?,What is Tianjin famous of?,0.968242,1.000000,1.000000,1.000000,0.993853,1,1
28021,Who will win this presidential elections 2016?,Who is going to win the 2016 presidential election?,0.970480,1.000000,0.479818,0.479818,0.700262,1,1
75211,What has been one of the best decisions of your life?,"What has been the best decision of your life, personally or professionally?",0.964307,1.000000,1.000000,1.000000,0.433231,1,0
41189,How can I get a complete list of all old Gmail accounts in my name?,How can I get a list of my Gmail addresses?,0.970995,1.000000,1.000000,1.000000,0.759572,1,1
42851,"Why do people ask such stupid questions on Quora, and more importantly why do people take the time to answer them?",Why is Quora filled with stupid questions and typically more stupid answers?,0.916662,0.000000,0.268885,0.268885,0.517858,1,0
91194,How can I learn something new everyday?,I want to learn something new everyday. What can it be and how can I change my attitude to do so?,0.969455,1.000000,0.430999,0.430999,0.557174,1,1
41038,"When designing a mobile app, what screen size and resolution should I create my designs?","When designing a mobile app, what screen size and resolution should I create?",0.993024,1.000000,1.000000,1.000000,0.899450,1,1
37315,What is the stupidest thing you've ever heard or seen?,What is the stupidest thing you have heard?,0.986440,1.000000,1.000000,1.000000,0.749619,1,1
28195,Why should Muslims support uniform civil code?,Why Muslims are against uniform civil code?,0.960962,1.000000,0.368032,0.368032,0.925975,1,1


In [78]:
type(y)

pandas.core.frame.DataFrame

In [79]:
embeddings_index = dict()
f = open('glove/glove.6B.300d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [80]:
vectorizer = CountVectorizer(binary=True, 
                             lowercase=True, max_df=0.9, max_features=10000)
vectorizer.fit(combined_qs)

word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}


In [81]:
count = 0
embedding_matrix = np.zeros((len(vectorizer.get_feature_names()), 300))
for index, word in enumerate(vectorizer.get_feature_names()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        count += 1
        
count

9881

In [82]:

tokenize = vectorizer.build_tokenizer()
preprocess = vectorizer.build_preprocessor()
 
def to_sequence(tokenizer, preprocessor, index, text):
    words = tokenizer(preprocessor(text))
    indexes = [index[word] for word in words if word in index]
    return indexes

#print(to_sequence(tokenize, preprocess, word2idx, "This is an important test!"))  # [2269, 4453]
X_all1_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in no_sw_q1s]
X_all2_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in no_sw_q2s]

N_FEATURES = len(vectorizer.get_feature_names())
MAX_SEQ_LEN = 30
X_all1_sequences = pad_sequences(X_all1_sequences, maxlen=MAX_SEQ_LEN, value=count)
X_all2_sequences = pad_sequences(X_all2_sequences, maxlen=MAX_SEQ_LEN, value=count)

sp_vals = X['spacy_sim'].values
sp_vals = sp_vals.reshape(sp_vals.shape[0],-1)

dot_vals = X['Dot'].values
dot_vals = dot_vals.reshape(dot_vals.shape[0],-1)

sp_vals = np.concatenate((sp_vals, dot_vals), axis=1)

q1_train,q1_holdout,q2_train,q2_holdout,y_train,y_holdout,df_train,df_holdout, sp_train, sp_holdout = \
    train_test_split(X_all1_sequences,X_all2_sequences,y,X,sp_vals,test_size=0.2)

In [83]:
print(q1_train.shape)

(80000, 30)


In [84]:
left_input = Input(shape=(MAX_SEQ_LEN,))
right_input = Input(shape=(MAX_SEQ_LEN,))
sp_input = Input(shape=(2,))

l2_reg = 0.0004
l1_reg = 0.00015

model = Sequential()

model.add(Embedding(len(vectorizer.get_feature_names()),
                    300,  
                    weights=[embedding_matrix], 
                    input_length=MAX_SEQ_LEN, trainable=True, embeddings_regularizer = L1L2(l1=0.0000025, l2=0.000005)))

model.add(GRU(256, dropout = 0.2, recurrent_dropout = 0.2, return_sequences=False, kernel_regularizer=L1L2(l1=l1_reg, l2=l2_reg)))

encoded_l = model(left_input)
encoded_r = model(right_input)


L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
L1_distance = L1_layer([encoded_l, encoded_r])

prediction = Dense(1,activation='sigmoid')(L1_distance)

siamModel = Model([left_input, right_input], outputs = prediction)


siamModel.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()
siamModel.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 300)           3000000   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               427776    
Total params: 3,427,776
Trainable params: 3,427,776
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
____________________________________________

In [85]:
callbacks = [EarlyStopping(monitor='val_loss', patience=4),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

siamModel.fit([q1_train, q2_train], np.ravel(y_train), validation_data = ([q1_holdout, q2_holdout], y_holdout), callbacks = callbacks, epochs=30, batch_size=32, verbose=2)


Instructions for updating:
Use tf.cast instead.
Train on 80000 samples, validate on 20000 samples
Epoch 1/30
 - 612s - loss: 1.7441 - acc: 0.6782 - val_loss: 1.0038 - val_acc: 0.7139
Epoch 2/30
 - 581s - loss: 0.8698 - acc: 0.7422 - val_loss: 0.8104 - val_acc: 0.7536
Epoch 3/30
 - 595s - loss: 0.7451 - acc: 0.7662 - val_loss: 0.7380 - val_acc: 0.7583
Epoch 4/30
 - 614s - loss: 0.6776 - acc: 0.7812 - val_loss: 0.6937 - val_acc: 0.7615
Epoch 5/30
 - 591s - loss: 0.6295 - acc: 0.7938 - val_loss: 0.6716 - val_acc: 0.7642
Epoch 6/30
 - 654s - loss: 0.5976 - acc: 0.8023 - val_loss: 0.6564 - val_acc: 0.7662
Epoch 7/30
 - 581s - loss: 0.5746 - acc: 0.8104 - val_loss: 0.6555 - val_acc: 0.7687
Epoch 8/30
 - 590s - loss: 0.5627 - acc: 0.8175 - val_loss: 0.6462 - val_acc: 0.7693
Epoch 9/30
 - 555s - loss: 0.5434 - acc: 0.8259 - val_loss: 0.6491 - val_acc: 0.7688
Epoch 10/30
 - 596s - loss: 0.5272 - acc: 0.8317 - val_loss: 0.6459 - val_acc: 0.7693
Epoch 11/30
 - 584s - loss: 0.5206 - acc: 0.8359 - 

In [86]:
siamModel.load_weights('best_model.h5')
y_prob_holdout = siamModel.predict([q1_holdout, q2_holdout])
y_pred_holdout = np.where(y_prob_holdout > 0.5, 1, 0)
print(confusion_matrix(y_holdout, y_pred_holdout))
print(f'accuracy: {accuracy_score(y_holdout, y_pred_holdout)}')
print(f'f1: {f1_score(y_holdout, y_pred_holdout)}')
print(f'log loss: {log_loss(y_holdout, y_prob_holdout)}')


[[10063  2423]
 [ 2191  5323]]
accuracy: 0.7693
f1: 0.6976408912188728
log loss: 0.48840396249395446


In [87]:
df_train.head()

,question1,question2,spacy_sim,equal_subj,equal_root,sim_what,Dot
47485,"When a man is in a romantic relationship and says 'I want to take care of you', what does this mean?",How can a woman take care of her man?,0.940455,0.334419,0.497859,0.497859,0.418945
19381,What are some things new employees should know going into their first day at Conn's?,What are some things new employees should know going into their first day at Stamps.com?,0.978826,1.000000,1.000000,1.000000,0.448317
98532,"I was In a relationship with a girl for nine months, yesterday she broke up with me saying that she loves her ex boyfriend?","I am dropper, I wasted my 4 months and now only 4 months are left and my concept is not clear now what can I do?",0.883310,0.509063,0.378756,0.378756,0.264089
9312,How can I get a proof of address when I live with my parent and don't pay any bills? I am 19.?,"I'm 17, and I was wondering if I should take pills to lower my sex drive…Should I?",0.948900,1.000000,0.350706,0.350706,0.026083
76236,What algorithm would you use to group points in a cluster from a given set of points?,Is there a standard algorithm to determine if a given point lies in any rectangle (out of a set of rectangles)?,0.946479,0.582355,0.382395,0.382395,0.263453


In [88]:
train_predictors = df_train.drop(['question1', 'question2'], axis=1)
train_predictors['siam_predict'] = siamModel.predict([q1_train, q2_train])
holdput_predictors = df_holdout.drop(['question1', 'question2'], axis=1)
holdout_predictors['siam_predict'] = siamModel.predict([q1_holdout, q2_holdout])
holdout_predictors.head()

,spacy_sim,equal_subj,equal_root,sim_what,Dot,siam_predict
6022,0.971348,1.000000,1.000000,1.000000,0.580913,0.230685
21535,0.751014,0.000000,0.192874,0.192874,0.000000,0.673163
20936,0.951501,1.000000,0.560781,0.560781,0.733327,0.229019
33016,0.832989,0.368162,0.746417,0.746417,0.007593,0.756297
28772,0.940367,0.000000,1.000000,1.000000,0.456935,0.294994


In [89]:
xgb = XGBClassifier()

In [90]:
xgb.fit(train_predictors, np.ravel(y_train))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [91]:
y_holdout_pred = xgb.predict(holdout_predictors)

In [92]:
print(confusion_matrix(y_holdout, y_holdout_pred))
print(f'accuracy: {accuracy_score(y_holdout, y_holdout_pred)}')
print(f'f1: {f1_score(y_holdout, y_holdout_pred)}')
print(f'log loss: {log_loss(y_holdout, y_prob_holdout)}')

[[10258  2228]
 [ 2599  4915]]
accuracy: 0.75865
f1: 0.6706693047690524
log loss: 0.48840396249395446


In [93]:
lr = LogisticRegression()

In [94]:
lr.fit(train_predictors, np.ravel(y_train))

/anaconda3/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [95]:
y_holdout_pred = lr.predict(holdout_predictors)

In [96]:
print(confusion_matrix(y_holdout, y_holdout_pred))
print(f'accuracy: {accuracy_score(y_holdout, y_holdout_pred)}')
print(f'f1: {f1_score(y_holdout, y_holdout_pred)}')
print(f'log loss: {log_loss(y_holdout, y_prob_holdout)}')

[[10241  2245]
 [ 2385  5129]]
accuracy: 0.7685
f1: 0.6890112842557765
log loss: 0.48840396249395446
